In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')

# Cell 2: Load Data
train_df = pd.read_csv('/path/to/train.csv')
test_df = pd.read_csv('/path/to/test.csv')

# Cell 3: Initialize stop words and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Cell 4: Define Enhanced Preprocessing Function
def enhanced_preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs, mentions, and hashtags
    text = re.sub(r"http\S+|www\S+|https\S+|@\S+|#\S+", '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'\W|\d', ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenize, remove stopwords, and stem words
    words = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to train and test datasets
train_df['cleaned_text'] = train_df['text'].apply(enhanced_preprocess_text)
test_df['cleaned_text'] = test_df['text'].apply(enhanced_preprocess_text)

# Cell 5: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=8000, ngram_range=(1, 2), max_df=0.8, min_df=5)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['cleaned_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['cleaned_text'])

# Cell 6: Split Data for Training and Validation
X = X_train_tfidf
y = train_df['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Cell 7: Set Up Grid Search for Hyperparameter Tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],          # Regularization parameter
    'solver': ['liblinear', 'saga'],       # Suitable solvers for small/medium datasets
    'penalty': ['l1', 'l2']                # L1 and L2 regularization
}

grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, scoring='f1', cv=5)
grid_search.fit(X_train, y_train)

# Retrieve best parameters and train final model
best_params = grid_search.best_params_
tuned_log_reg = LogisticRegression(**best_params, max_iter=1000)
tuned_log_reg.fit(X_train, y_train)

# Cell 8: Validate the Tuned Model
y_val_pred_tuned = tuned_log_reg.predict(X_val)
tuned_f1 = f1_score(y_val, y_val_pred_tuned)
print("Validation F1 Score after tuning:", tuned_f1)

# Cell 9: Generate Predictions on the Test Set
test_predictions = tuned_log_reg.predict(X_test_tfidf)

# Cell 10: Prepare Submission File
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions
})

# Save the submission file
submission_df.to_csv('/path/to/enhanced_logistic_regression_submission.csv', index=False)
print("Submission file saved as 'enhanced_logistic_regression_submission.csv'")
